In [1]:
import cianparser
# import pandas as pd
import requests
# from bs4 import BeautifulSoup
import cloudscraper
# from datetime import datetime
# import pytz
import time
import random
# from datetime import timedelta
from PIL import Image
from io import BytesIO
from tqdm import trange, tqdm
from IPython import display

replace(',','.') для площадей (done)

вывод времени на ожидание (done)

лок на 15 картинок (done)

починить высоту потолков (done, but need to check)

сделать одну жирную функцию, которая сразу сохраняет csv

попробовать поставить билиотеку вручную и исправить сити нейборс на то, что бы искало или добавить в список городов айди ленобласти (done)

оптимизировать процесс сбора фоток, наверное сократить до 10 + убрать/уменьшить паузу на сбор инфы вцелом(done)

In [2]:
scraper = cloudscraper.create_scraper()
scraper.headers = {'Accept-Language': 'en'}

In [20]:
def parse_offer(scraper,url):
    status = 1
    page = scraper.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    if soup.text.find("Captcha") > 0:
            print('Капча')
            status = 0
    elif soup.text.find('Объявление снято с публикации') > 0:
          print('Объявление снято с публикации')
          status = 0
    elif page.status_code == 200:
          print('Страница получена')
    else:
          print('Что-то пошло не так ({})'.format(page.status_code)) 
          status = 0 
    return soup,status

In [4]:
def get_date_of_post(soup):
    date_offer_full = soup.find('div',class_ = "a10a3f92e9--item--qLmCs").find('span')
    date_offer_full = date_offer_full.text
    date_offer = date_offer_full[date_offer_full.find(':')+2:]
    now = datetime.now(pytz.timezone('Europe/Moscow'))
    date_list = date_offer.split(',')
    if date_list[0] == 'вчера':
        vchera = now - timedelta(days=1)
        vchera = vchera.strftime("%d/%m/%Y")
        date_list[0] = vchera
    elif date_list[0] == 'сегодня':
        today = now.strftime("%d/%m/%Y")
        date_list[0] = today
    else:
        date_0 = date_list[0].split(' ')
        if date_0[1] == 'ноя':
            date_0 = '{}/11/2023'.format(date_0[0])
            date_list[0] = date_0
        if date_0[1] == 'дек':
            date_0 = '{}/12/2023'.format(date_0[0])
            date_list[0] = date_0
        if date_0[1] == 'янв':
            date_0 = '{}/1/2024'.format(date_0[0])
            date_list[0] = date_0
        if date_0[1] == 'фев':
            date_0 = '{}/2/2024'.format(date_0[0])
            date_list[0] = date_0
    date_offer = ' '.join(date_list)
    return(date_offer)

In [5]:
about_flat_info = 'a10a3f92e9--color_black_100--Ephi7 a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5'
about_flat_type_info = 'a10a3f92e9--color_gray60_100--mYFjS a10a3f92e9--lineHeight_6u--cedXD a10a3f92e9--fontWeight_normal--JEG_c a10a3f92e9--fontSize_16px--QNYmt a10a3f92e9--display_block--KYb25 a10a3f92e9--text--e4SBY a10a3f92e9--text_letterSpacing__0--cQxU5'

In [6]:
HOUSE_INFO = ['Жилая площадь','Площадь кухни','Высота потолков','Планировка','Санузел','Балкон/лоджия','Вид из окон','Ремонт','Год постройки','Тип дома','Аварийность','Парковка']
len(HOUSE_INFO)

12

In [7]:
def get_house_info(soup,house_info_dict,i):
    len_home_info = len(soup.select('div[data-name="OfferSummaryInfoItem"]'))
    temp_info_dict = {}
    for h in range(len_home_info):
        type_info = soup.select('div[data-name="OfferSummaryInfoItem"]')[h].find('span', class_ = about_flat_type_info).text
        info = soup.select('div[data-name="OfferSummaryInfoItem"]')[h].find('span', class_ = about_flat_info).text
        if info != 'Нет информации' and type_info != 'Общая площадь':
            if type_info == 'Высота потолков':
                 if info.find(',') != -1:
                    info = info[0] + '.' + info[2]
                 else:
                     info = info[0]
            if type_info in ['Жилая площадь','Площадь кухни']:
                info = info.replace(',','.')
            if type_info == 'Санузел':
                 if info.find(',') != -1:
                    toilet = info.split(',')
                    info = int(toilet[0][0]) + int(toilet[1][1])
                 else:
                    info = int(info[0])
            temp_info_dict[type_info] = info
        # print('{}: {}'.format(type_info,info))
    for key in list(temp_info_dict.keys()):
            if key in HOUSE_INFO:
                house_info_dict[key][i] = temp_info_dict[key]

In [8]:
FEATURES = ['Холодильник','Стиральная машина','Телевизор','Ванна','Мебель на кухне','Посудомоечная машина','Кондиционер','Интернет','Душевая кабина','Мебель в комнатах']
len(FEATURES)

10

In [9]:
def get_features(soup,features_dict,i):
    features = soup.select('div[data-name="FeaturesItem"]')
    for f in features:
        features_dict[f.text][i] = 1

In [10]:
# for i in [0,0,0,2,1,4]:
#     try:
#         ans = 5/i
#         break
#     except ZeroDivisionError:
#         pass

# print(ans)

In [11]:
def get_photos(soup, id_):
    photos = soup.select('img[data-name="ThumbComponent"]')
    len_photos = len(photos)
    if len_photos > 10:
        len_photos = 10
    for i in trange(len_photos, desc= 'photos', colour= 'CYAN'):
        ph_link = photos[i].get('src')
        ph_link = ph_link[:-5] + '1' +ph_link[-4:]
        for i in range(5):
            try:
                r = scraper.get(ph_link)
                im = Image.open(BytesIO(r.content))
                im = im.resize((300,300), Image.LANCZOS)
                im.save('photos/{}_{}.jpg'.format(id_,i))
            except ConnectionError:
                time.sleep(3)
                pass
        # im = Image.open(BytesIO(r.content))
        # im = im.resize((300,300), Image.LANCZOS)
        # im.save('photos/{}_{}.jpg'.format(id_,i))
        rnd = 3*random.random()
        time.sleep(5+rnd)

In [12]:
def create_h_dict(links):
    house_info_dict = {}
    house_info_dict['id'] = [0 for i in range(len(links))]
    for item in HOUSE_INFO:
        house_info_dict[item] = [None for i in range(len(links))]
    return(house_info_dict)

In [13]:
def create_f_dict(links):
    features_dict = {}
    features_dict['id'] = [0 for i in range(len(links))]
    for item in FEATURES:
        features_dict[item] = [0 for i in range(len(links))]
    return(features_dict)

In [14]:
def create_new_df(data):
    df2 = pd.DataFrame(data)
    df2['underground'] = df2['underground'].apply(lambda x: None if x == '' else x)
    df2 = df2.drop(['author','author_type','deal_type','commissions','accommodation_type'],axis = 1)
    try:
        df2 = df2.drop('residential_complex', axis = 1)
    except KeyError:
        pass
    return(df2)
    

In [15]:
def get_old_df(LOCATION,ROOM):
    if LOCATION == 'Ленобласть':
        city = 'LEN_OBL'
    else:
        city = 'SPB'
    df_old = pd.read_csv('cian_data_r_{}_c_{}.csv'.format(ROOM,city),index_col=0)
    return(df_old,city)

In [16]:
def update_dfs(df_old,df2):
    links = df2['link']
    links_to_drop = []
    curr_time = datetime.now(pytz.timezone('Europe/Moscow')).strftime("%d/%m/%Y %H:%M")
    for i in range(len(links)):
        url = links[i]
        old_links = list(df_old['link'])
        if url in old_links:
            if df_old[df_old['link'] == url]['price_per_month'].item() == df2['price_per_month'][i].item():
                df_old.loc[df_old['link'] == url,'time_parse'] = curr_time
                links_to_drop.append(url)
            else: 
                df_old = df_old.drop(df_old[df_old['link'] == url].index, axis = 0).reset_index(drop = True)
    df2 = df2.drop(df2[df2['link'].isin(links_to_drop)].index, axis = 0).reset_index(drop = True)
    print('Объявлений уже в данных: {}'.format(len(links_to_drop)))
    return df_old, df2
    

In [24]:
# ROOMS = ['studio',1,2,3,4,5]
ROOMS = [1,2,3,4,5]
LOCATIONS = ['Санкт-Петербург','Ленобласть']
# LOCATIONS = ['Ленобласть']
ADD_SETTINGS = {'sort_by' :'creation_data_from_newer_to_older'}

начать с двойки

In [26]:
#GIGA_MEGA_PARSE
for LOCATION in LOCATIONS:
    for ROOM in ROOMS:
        for page in range(1,20): #55 MAX
            display.clear_output(wait=False)
            data = cianparser.parse(deal_type="rent_long",accommodation_type="flat",location=LOCATION,rooms=ROOM,additional_settings=ADD_SETTINGS,
            start_page=page,
            end_page=page)

            #скип цикла, если страницы кончились
            if len(data) == 0:
                continue
            df2 = create_new_df(data)
            print('Найдено объявлений: {}'.format(len(df2)))
            df_old, city = get_old_df(LOCATION,ROOM)
            df_old, df2 = update_dfs(df_old,df2)    
            links = df2['link']
            print('Объявлений на обработку: {}'.format(len(links)))
            house_info_dict = create_h_dict(links)
            features_dict = create_f_dict(links)
            ids_ = []
            date_info = []
            time_parse = []

            for i in range(len(links)):
                print('-'*20,'{} | {}'.format(ROOM,city),'-'*20) 
                url = links[i]
                print('{}/{} '.format(i+1,len(links)),url)
                id_ = url.split('/')[-2]
                ids_.append(id_)
                scraper = cloudscraper.create_scraper()
                rnd_name = random.choice(['dasha','masha','tosha','sasha','vasya','kolya','anna','fedya','bob','mark','grisha'])
                rnd_dig = random.randint(100,9000)
                scraper.headers = {'Accept-Language': 'en','User-Agent': '{}{}@gmail.com'.format(rnd_name,rnd_dig)}
                soup, satatus = parse_offer(scraper,url)
                #скип цикла, если страница не алё
                if satatus == 0:
                    date_info.append(0)
                    time_parse.append(0)
                    continue
                with tqdm(total=3,desc = 'info', colour = 'CYAN') as pbar:
                    date_offer = get_date_of_post(soup)
                    date_info.append(date_offer)
                    time_parse.append(datetime.now(pytz.timezone('Europe/Moscow')).strftime("%d/%m/%Y %H:%M"))
                    pbar.update(1)
                    get_house_info(soup,house_info_dict,i)
                    house_info_dict['id'][i] = id_
                    pbar.update(1)
                    get_features(soup,features_dict,i)
                    features_dict['id'][i] = id_
                    pbar.update(1)
                get_photos(soup,id_)
                rnd = 3*random.random()
                time.sleep(5+rnd)
                
            
            df_h = pd.DataFrame(house_info_dict)
            df_f = pd.DataFrame(features_dict)
            df2.insert(0,'id',ids_)
            df2.insert(1,'time_post',date_info)
            df2.insert(2,'time_parse',time_parse)
            df3_j1 = df2.join(df_f.set_index('id'),on = 'id')
            df3_j2 = df3_j1.join(df_h.set_index('id'),on = 'id')
            df4 = pd.concat([df_old,df3_j2]).reset_index(drop = True)
            df4.to_csv('cian_data_r_{}_c_{}.csv'.format(ROOM,city))


                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&region=2&offer_type=flat&deal_type=rent&room1=1&with_neighbors=0&type=4&sort=creation_date_desc


In [ ]:
# soup,status =  parse_offer(scraper,'https://spb.cian.ru/rent/flat/295015271/')

Объявление снято с публикации


In [ ]:
# #GIGA_MEGA_PARSE
# for LOCATION in LOCATIONS:
#     for ROOM in ROOMS:
        
#         data = cianparser.parse(deal_type="rent_long",accommodation_type="flat",location=LOCATION,rooms=ROOM,additional_settings=ADD_SETTINGS,
#         start_page=1,
#         end_page=1)

#         df2 = create_new_df(data)
#         print('Найдено объявлений: {}'.format(len(df2)))
#         df_old, city = get_old_df(LOCATION,ROOM)
#         df_old, df2 = update_dfs(df_old,df2)    
#         links = df2['link']
#         print('Объявлений на обработку: {}'.format(len(links)))
#         house_info_dict = create_h_dict(links)
#         features_dict = create_f_dict(links)
#         ids_ = []
#         date_info = []
#         time_parse = []

#         for i in range(len(links)):
#             print('-'*20,'{} | {}'.format(ROOM,city),'-'*20) # попробовать 12
#             url = links[i]
#             print('{}/{} '.format(i+1,len(links)),url)
#             id_ = url.split('/')[-2]
#             ids_.append(id_)
#             soup, satatus = parse_offer(scraper,url)
#             if satatus == 0:
#                 continue
#             with tqdm(total=3,desc = 'info') as pbar:
#                 date_offer = get_date_of_post(soup)
#                 date_info.append(date_offer)
#                 time_parse.append(datetime.now(pytz.timezone('Europe/Moscow')).strftime("%d/%m/%Y %H:%M"))
#                 pbar.update(1)
#                 get_house_info(soup,house_info_dict,i)
#                 house_info_dict['id'][i] = id_
#                 pbar.update(1)
#                 get_features(soup,features_dict,i)
#                 features_dict['id'][i] = id_
#                 pbar.update(1)
#             get_photos(soup,id_)
#             rnd = 3*random.random()
#             time.sleep(5+rnd)
            
        
#         df_h = pd.DataFrame(house_info_dict)
#         df_f = pd.DataFrame(features_dict)
#         df2.insert(0,'id',ids_)
#         df2.insert(1,'time_post',date_info)
#         df2.insert(2,'time_parse',time_parse)
#         df3_j1 = df2.join(df_f.set_index('id'),on = 'id')
#         df3_j2 = df3_j1.join(df_h.set_index('id'),on = 'id')
#         df4 = pd.concat([df_old,df3_j2]).reset_index(drop = True)
#         df4.to_csv('cian_data_r_{}_c_{}.csv'.format(ROOM,city))



                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&region=2&offer_type=flat&deal_type=rent&room1=1&with_neighbors=0&type=4&sort=creation_date_desc
 1 | 1 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 100 %. Average price: 32 428 rub

The collection of information from the pages with list of announcements is completed
Total number of parsed announcements: 28. Average price per month: 32 428 rub

Найдено объявлений: 28
Объявлений уже в данных: 0
Объявлений на обработку: 28
-------------------- 1 | SPB --------------------
1/28  https://spb.cian.ru/rent/flat/295883550/
Страница получена


info: 100%|██████████| 3/3 [00:00<00:00, 15.15it/s]
photos: 0it [00:00, ?it/s]


: 

----

timing - примерно обьявление в минуту

In [ ]:
a = 1/0

ZeroDivisionError: division by zero

In [ ]:
# data = cianparser.parse(
#     deal_type="rent_long",
#     accommodation_type="flat",
#     location=LOCATION,
#     rooms=ROOM,
#     start_page=1,
#     end_page=4,
#     additional_settings={'sort_by' :'creation_data_from_newer_to_older'}
# )

In [ ]:
# df2 = pd.DataFrame(data)
# print(len(df2))

In [ ]:
df2

In [ ]:
# df2['underground'] = df2['underground'].apply(lambda x: None if x == '' else x)

In [ ]:
df2

In [ ]:
# df2 = df2.drop(['author','author_type','deal_type','commissions','accommodation_type'],axis = 1)

In [ ]:
# if LOCATION == 'Ленобласть':
#     CITY = 'LEN_OBL'
# else:
#     CITY = 'SPB'

In [ ]:
# df_old = pd.read_csv('cian_data_r_{}_c_{}.csv'.format(ROOM,CITY))

In [ ]:
# links = df2['link']

In [ ]:
# a = ['1','2','3','4']
# a.pop(1)
# a

['1', '3', '4']

In [ ]:
# ids_ = []
# links_to_drop = []
# curr_time = datetime.now(pytz.timezone('Europe/Moscow')).strftime("%d/%m/%Y %H:%M")
# for i in range(len(links)):
#     url = links[i]
#     id_ = url.split('/')[-2]
#     if id_ in df_old['id']:
#         if df_old[df_old['id'] == id_]['price'] == df2['price'][i]:
#             df_old[df_old['id'] == id_]['time_parse'] = curr_time
#             links_to_drop.append(i)
#         else: 
#             df = df_old.drop(df_old[df_old['id'] == id_], axis = 0)
# links = links.drop(links_to_drop,axis = 0)
    

In [ ]:
# house_info_dict = {}
# house_info_dict['id'] = [0 for i in range(len(links))]
# for item in HOUSE_INFO:
#     house_info_dict[item] = [None for i in range(len(links))]

In [ ]:
# features_dict = {}
# features_dict['id'] = [0 for i in range(len(links))]
# for item in FEATURES:
#     features_dict[item] = [0 for i in range(len(links))]

In [ ]:
# ids_ = []
# date_info = []
# time_parse = []
# for i in range(len(links)):
#     url = links[i]
#     print('{}/{} '.format(i+1,len(links)),url)
#     id_ = url.split('/')[-2]
#     ids_.append(id_)
#     soup = parse_offer(scraper,url)
#     with tqdm(total=3,desc = 'info') as pbar:
#         date_offer = get_date_of_post(soup)
#         date_info.append(date_offer)
#         time_parse.append(datetime.now(pytz.timezone('Europe/Moscow')).strftime("%d/%m/%Y %H:%M"))
#         pbar.update(1)
#         get_house_info(soup,house_info_dict,i)
#         house_info_dict['id'][i] = id_
#         pbar.update(1)
#         get_features(soup,features_dict,i)
#         features_dict['id'][i] = id_
#         pbar.update(1)
#     get_photos(soup,id_)
#     rnd = 3*random.random()
#     time.sleep(5+rnd)
#     print('------------------------------------------')

In [ ]:
# df_h = pd.DataFrame(house_info_dict)
# df_h.head()

In [ ]:
# df_f = pd.DataFrame(features_dict)
# df_f.head()

In [ ]:
# df2.insert(0,'id',ids_)
# df2.insert(1,'time_post',date_info)
# df2.insert(2,'time_parse',time_parse)

In [ ]:
# df2.head()

In [ ]:
# df3_j1 = df2.join(df_f.set_index('id'),on = 'id')
# df3_j1.head()

In [ ]:
# df3_j2 = df3_j1.join(df_h.set_index('id'),on = 'id')
# df3_j2.head()

In [ ]:
# save_time = datetime.now(pytz.timezone('Europe/Moscow')).strftime("%d-%m-%Y_%H-%M")
# df3_j2.to_csv('cian_data_r_{}_d_{}.csv'.format(ROOM,save_time))